# Лабораторная работа №2
# Поиск ассоциативных правил

Поиск ассоциативных правил представляет собой весьма часто применяемый метод выявления зависимостей между записями в больших наборах данных.

Ассоциативные правила обычно используются при анализе продуктовых корреляций (Market Basket Analysis) и выглядят примерно так: «если в чеке клиента есть молоко, то в 80% случаев там будет и хлеб». 

## Набор транзакций Instacart

Компания Instacart (сайт https://www.instacart.com) – розничный продавец овощей в США и Канаде (слоган «Groceries delivered in as little as 1 hour»). 

Для исследователей на сайте Instacart был выложен набор данных “The Instacart Online Grocery Shopping Dataset 2017”, адрес набора
https://www.instacart.com/datasets/grocery-shopping-2017.

Набор данных анонимизирован и состоит из следующих таблиц.

`orders` (3.4m rows, 206k users):
* `order_id`: order identifier
* `user_id`: customer identifier
* `eval_set`: which evaluation set this order belongs in (see `SET` described below)
* `order_number`: the order sequence number for this user (1 = first, n = nth)
* `order_dow`: the day of the week the order was placed on
* `order_hour_of_day`: the hour of the day the order was placed on
* `days_since_prior`: days since the last order, capped at 30 (with NAs for `order_number` = 1)

`products` (50k rows):
* `product_id`: product identifier
* `product_name`: name of the product
* `aisle_id`: foreign key
* `department_id`: foreign key

`aisles` (134 rows):
* `aisle_id`: aisle identifier
* `aisle`: the name of the aisle

`deptartments` (21 rows):
* `department_id`: department identifier
* `department`: the name of the department

`order_products__SET` (30m+ rows):
* `order_id`: foreign key
* `product_id`: foreign key
* `add_to_cart_order`: order in which each product was added to cart
* `reordered`: 1 if this product has been ordered by this user in the past, 0 otherwise

где `SET` - один из следующих наборов (`eval_set` в таблице `orders`):
* `"prior"`: orders prior to that users most recent order (~3.2m orders)
* `"train"`: training data supplied to participants (~131k orders)
* `"test"`: test data reserved for machine learning competitions (~75k orders)

Для работы с набором данных Instacart будем использовать встроенную в Python СУБД `sqlite`.

## SQL

База данных (БД) - это данные, которые хранятся в соответствии с определенной схемой, описывающей соотношения между данными.

Язык БД SQL (structured query language) - используется для описания структуры БД, управления данными (добавление, изменение, удаление, получение), управления правами доступа к БД и ее объектам, управления транзакциями.

Система управления базами данных (СУБД) - это программные средства, которые дают возможность управлять БД, поддерживая соответствующий язык (языки) для управления БД.

Язык SQL подразделяется на следующие категории:

`DDL` (Data Definition Language) - язык описания данных
* `CREATE` - создание новой таблицы, СУБД, схемы
* `ALTER` - изменение существующей таблицы, колонки 
* `DROP` - удаление существующих объектов из СУБД

`DML` (Data Manipulation Language) - язык манипулирования данными 
* `SELECT` - выбор данных
* `INSERT` - добавление новых данных
* `UPDATE` - обновление существующих данных 
* `DELETE` - удаление данных

`DCL` (Data Control Language) - язык определения доступа к данным 
* `GRANT` - предоставление пользователям разрешения на чтение/запись определенных объектов в СУБД
* `REVOKE` - отзыв ранее предоставленных разрешений

`TCL` (Transaction Control Language) - язык управления транзакциями
* `COMMIT` - применение транзакции
* `ROLLBACK` - откат всех изменений, сделанных в текущей транзакции


## SQLite

SQLite — компактная встраиваемая СУБД с открытым исходным кодом, написанная на языке C.

SQLite не использует парадигму клиент-сервер, а именно, SQLite представляет собой библиотеку, с которой компонуется программа, использующая SQLite. SQLite является составной частью Python.

### Утилита sqlite3

В комплект поставки SQLite входит утилита sqlite3 для работы с SQLite в командной строке. С помощью sqlite3 можно вручную выполнять команды SQL.

Для доступа к базе данных Instacart при помощи sqlite3 нужно выполнить следующую команду в окне терминала:

`sqlite3 instacart.db`


В sqlite3 можно выполнять команды SQL или так называемые метакоманды (или dot-команды).

К метакомандам относятся несколько специальных команд для работы с SQLite. Они относятся только к утилите sqlite3, а не к SQL языку. В конце этих команд ; ставить не нужно.
Примеры метакоманд:

* .help - подсказка со списком всех метакоманд 
* .exit или .quit - выход из сессии sqlite3
* .databases - показывает присоединенные БД 
* .tables - показывает доступные таблицы

При выполнении в sqlite3 команды SQL ее нужно завершить символом `;`, например:

`select count(*) from orders;`

### Модуль sqlite3

Для работы с SQLite в Python используется модуль sqlite3.

Объект `Connection` - это подключение к конкретной БД. 
Пример создания подключения:

In [1]:
import numpy as np
import pandas as pd
import sqlite3

conn = sqlite3.connect('temp.db')

In [2]:
type(conn)

sqlite3.Connection

Если база данных temp.db не существует, то она будет создана. После создания соединения надо создать объект Cursor - это основной способ работы с БД. Создается курсор из соединения с БД:


In [3]:
cursor = conn.cursor()

type(cursor)

sqlite3.Cursor

### Выполнение команд SQL

Для выполнения команд SQL в модуле есть несколько методов:

* `execute()` - метод для выполнения одного выражения SQL
* `executemany()` - метод позволяет выполнить одно выражение SQL для последовательности параметров (или для итератора)
* `executescript()` - метод позволяет выполнить несколько выражений SQL за один раз

#### Метод execute()

Метод execute() позволяет выполнить одну команду SQL (при условии, что уже созданы соединение и курсор). 

Создадим таблицу user с помощью метода execute():

In [4]:
#cursor.execute("""drop table user""")

cursor.execute("""create table user 
    (userid text not NULL primary key, 
    username text, 
    room text, 
    phone text)""")

OperationalError: table user already exists

Выражения SQL могут быть параметризированы - вместо данных можно подставлять специальные значения. За счет этого можно использовать одну и ту же команду SQL для передачи разных данных.

Например, таблицу user нужно заполнить данными из списка data:

In [5]:
data = [
    ('001', 'Иванов И.И.',  '123', '(985)1234567'), 
    ('002', 'Петров П.П.',  '234', '(903)9876543'), 
    ('003', 'Сидоров С.С.', '345', '(495)1357900')] 

Для этого можно использовать запрос вида:

In [6]:
query = "INSERT into user values (?, ?, ?, ?)"

Знаки вопроса в команде используются для подстановки данных, которые будут передаваться методу execute.

Теперь можно передать данные таким образом:

In [7]:
for row in data:
    cursor.execute(query, row)

IntegrityError: UNIQUE constraint failed: user.userid

Второй аргумент, который передается методу execute, должен быть кортежем. Если нужно передать кортеж с одним элементом, используется запись вида (value, ).

Чтобы изменения были применены, нужно выполнить commit() (обратите внимание, что метод commit() вызывается у соединения):

In [8]:
conn.commit()

Обратимся к сохраненным данным:

In [9]:
for row in cursor.execute("SELECT * FROM user"):
    print(row)

('001', 'Иванов И.И.', '123', '(985)1234567')
('002', 'Петров П.П.', '234', '(903)9876543')
('003', 'Сидоров С.С.', '345', '(495)1357900')
('004', 'Васильев В.В.', '456', '(901)1112233')
('005', 'Сергеев С.С.', '567', '(905)1231212')


#### Метод executemany()

Метод executemany() позволяет выполнить одну команду SQL для последовательности параметров (или для итератора).

С помощью метода executemany() в таблицу user можно добавить аналогичный список данных одной командой.

Например, в таблицу user надо добавить данные из списка data2:

In [10]:
data2 = [
    ('004', 'Васильев В.В.', '456', '(901)1112233'),
    ('005', 'Сергеев С.С.',  '567', '(905)1231212')]

Для этого нужно использовать аналогичный запрос вида:

In [11]:
query = "INSERT into user values (?, ?, ?, ?)"

Теперь можно передать данные методу executemany():

In [12]:
cursor.executemany(query, data2) 

conn.commit()

IntegrityError: UNIQUE constraint failed: user.userid

Метод executemany() подставил соответствующие кортежи в команду SQL, и все данные добавились в таблицу.

In [13]:
for row in cursor.execute("SELECT * FROM user"):
    print(row)

('001', 'Иванов И.И.', '123', '(985)1234567')
('002', 'Петров П.П.', '234', '(903)9876543')
('003', 'Сидоров С.С.', '345', '(495)1357900')
('004', 'Васильев В.В.', '456', '(901)1112233')
('005', 'Сергеев С.С.', '567', '(905)1231212')


#### Метод executescript()

Метод executescript() позволяет выполнить несколько выражений SQL за один раз:

In [14]:
cursor.executescript('''
delete from user;
drop table user;
''')
conn.close()

### Получение результатов запроса

Для получения результатов запроса в sqlite3 есть несколько способов:

* использование методов fetch...() - в зависимости от метода возвращаются одна, несколько или все строки
* использование курсора как итератора - возвращается итератор

#### Метод fetchone()

Метод fetchone() возвращает одну строку данных.

Пример получения информации из базы данных instacart.db:

In [15]:
conn = sqlite3.connect('instacart.db')

cursor = conn.cursor()

cursor.execute('select * from departments') 

cursor.fetchone()

('1', 'frozen')

Обратите внимание, что хотя запрос SQL подразумевает, что запрашивалось всё содержимое таблицы, метод fetchone() вернул только одну строку.

Если повторно вызвать метод, он вернет следующую строку:

In [16]:
cursor.fetchone()

('2', 'other')

Аналогичным образом метод будет возвращать следующие строки. После обработки всех строк метод начинает возвращать None.

За счет этого метод можно использовать в цикле, например, так:

In [17]:
cursor.execute('select * from departments')

while True:
    next_row = cursor.fetchone() 
    if next_row:
        print(next_row) 
    else:
        break

('1', 'frozen')
('2', 'other')
('3', 'bakery')
('4', 'produce')
('5', 'alcohol')
('6', 'international')
('7', 'beverages')
('8', 'pets')
('9', 'dry goods pasta')
('10', 'bulk')
('11', 'personal care')
('12', 'meat seafood')
('13', 'pantry')
('14', 'breakfast')
('15', 'canned goods')
('16', 'dairy eggs')
('17', 'household')
('18', 'babies')
('19', 'snacks')
('20', 'deli')
('21', 'missing')


#### Метод fetchmany()

Метод fetchmany() возвращает список строк данных. Синтаксис метода:

`cursor.fetchmany([size=cursor.arraysize])`

С помощью параметра size можно указывать, какое количество строк возвращается. По умолчанию параметр size равен значению cursor.arraysize:

In [18]:
print(cursor.arraysize)

1


Например, таким образом можно возвращать по три строки из запроса:

In [19]:
cursor.execute('select * from aisles')

while True:
    three_rows = cursor.fetchmany(3) 
    if three_rows:
        print(three_rows) 
    else:
        break

[('1', 'prepared soups salads'), ('2', 'specialty cheeses'), ('3', 'energy granola bars')]
[('4', 'instant foods'), ('5', 'marinades meat preparation'), ('6', 'other')]
[('7', 'packaged meat'), ('8', 'bakery desserts'), ('9', 'pasta sauce')]
[('10', 'kitchen supplies'), ('11', 'cold flu allergy'), ('12', 'fresh pasta')]
[('13', 'prepared meals'), ('14', 'tofu meat alternatives'), ('15', 'packaged seafood')]
[('16', 'fresh herbs'), ('17', 'baking ingredients'), ('18', 'bulk dried fruits vegetables')]
[('19', 'oils vinegars'), ('20', 'oral hygiene'), ('21', 'packaged cheese')]
[('22', 'hair care'), ('23', 'popcorn jerky'), ('24', 'fresh fruits')]
[('25', 'soap'), ('26', 'coffee'), ('27', 'beers coolers')]
[('28', 'red wines'), ('29', 'honeys syrups nectars'), ('30', 'latino foods')]
[('31', 'refrigerated'), ('32', 'packaged produce'), ('33', 'kosher foods')]
[('34', 'frozen meat seafood'), ('35', 'poultry counter'), ('36', 'butter')]
[('37', 'ice cream ice'), ('38', 'frozen meals'), ('39

Метод выдает нужное количество строк, а если строк осталось меньше, чем параметр size, то оставшиеся строки.

#### Метод fetchall()

Метод fetchall() возвращает все строки в виде списка:

In [20]:
cursor.execute('select * from departments')

cursor.fetchall()

[('1', 'frozen'),
 ('2', 'other'),
 ('3', 'bakery'),
 ('4', 'produce'),
 ('5', 'alcohol'),
 ('6', 'international'),
 ('7', 'beverages'),
 ('8', 'pets'),
 ('9', 'dry goods pasta'),
 ('10', 'bulk'),
 ('11', 'personal care'),
 ('12', 'meat seafood'),
 ('13', 'pantry'),
 ('14', 'breakfast'),
 ('15', 'canned goods'),
 ('16', 'dairy eggs'),
 ('17', 'household'),
 ('18', 'babies'),
 ('19', 'snacks'),
 ('20', 'deli'),
 ('21', 'missing')]

Важный аспект работы метода - он возвращает все оставшиеся строки.

То есть, если до метода fetchall() использовался, например, метод fetchone(), то метод fetchall() вернет оставшиеся строки запроса.

#### Cursor как итератор

Если нужно построчно обрабатывать результирующие строки, лучше использовать курсор как итератор. При этом не нужно использовать методы fetch.

При использовании методов execute возвращается курсор. А, так как курсор можно использовать как итератор, можно использовать его, например, в цикле for:

In [21]:
result = cursor.execute('select * from departments')

for row in result: 
    print(row)

('1', 'frozen')
('2', 'other')
('3', 'bakery')
('4', 'produce')
('5', 'alcohol')
('6', 'international')
('7', 'beverages')
('8', 'pets')
('9', 'dry goods pasta')
('10', 'bulk')
('11', 'personal care')
('12', 'meat seafood')
('13', 'pantry')
('14', 'breakfast')
('15', 'canned goods')
('16', 'dairy eggs')
('17', 'household')
('18', 'babies')
('19', 'snacks')
('20', 'deli')
('21', 'missing')


Аналогичный вариант отработает и без присваивания переменной:

In [22]:
for row in cursor.execute('select * from departments'): 
    print(row)

('1', 'frozen')
('2', 'other')
('3', 'bakery')
('4', 'produce')
('5', 'alcohol')
('6', 'international')
('7', 'beverages')
('8', 'pets')
('9', 'dry goods pasta')
('10', 'bulk')
('11', 'personal care')
('12', 'meat seafood')
('13', 'pantry')
('14', 'breakfast')
('15', 'canned goods')
('16', 'dairy eggs')
('17', 'household')
('18', 'babies')
('19', 'snacks')
('20', 'deli')
('21', 'missing')


### Использование sqlite3 без создания курсора


Методы execute доступны и в объекте Connection, и в объекте Cursor, а методы fetch доступны только в объекте Cursor.
При использовании методов execute с объектом Connection курсор возвращается как результат выполнения метода execute. Его можно использовать как итератор и получать данные без методов fetch. За счет этого при работе с модулем sqlite3 можно не создавать курсор.

In [23]:
data = [
    ('001', 'Иванов И.И.', '123', '(985)1234567'),
    ('002', 'Петров П.П.', '234', '(903)9876543'),
    ('003', 'Сидоров С.С.', '345', '(495)1357900'),
    ('004', 'Васильев В.В.', '456', '(901)1112233'),
    ('005', 'Сергеев С.С.', '567', '(905)1231212'),
]
conn2 = sqlite3.connect('temp.db') 
conn2.execute('''
    create table user 
        (userid text not NULL primary key, 
        username text, 
        room text, 
        phone text)
    '''
)
query = 'INSERT into user values (?, ?, ?, ?)' 
conn2.executemany(query, data)
conn2.commit()
for row in conn2.execute('select * from user'):
    print(row) 
#conn2.close()

('001', 'Иванов И.И.', '123', '(985)1234567')
('002', 'Петров П.П.', '234', '(903)9876543')
('003', 'Сидоров С.С.', '345', '(495)1357900')
('004', 'Васильев В.В.', '456', '(901)1112233')
('005', 'Сергеев С.С.', '567', '(905)1231212')


### Обработка исключений

Посмотрим на пример использования метода execute при возникновении ошибки.
В таблице user поле mac должно быть уникальным. И, если попытаться записать пересекающийся MAC-адрес, возникнет ошибка:

In [24]:
query = "INSERT into user values ('005', 'Антонов А.А.', '678', '(979)9779977')"

conn2.execute(query)

IntegrityError: UNIQUE constraint failed: user.userid

Соответственно, можно перехватить исключение:

In [25]:
try:
    conn2.execute(query)
except sqlite3.IntegrityError as e:
    print("Произошла ошибка: ", e)
    
conn2.close()   

Произошла ошибка:  UNIQUE constraint failed: user.userid


Обратите внимание, что надо перехватывать исключение sqlite3.IntegrityError, а не IntegrityError.

## SQL запросы к базе данных Instacart

SQL запросы к базе данных Instacart будут предусматривать соединение различных таблиц базы данных. Например, подсчитаем количество заказов в разбивке по департаментам:

In [26]:
for row in cursor.execute("""
        SELECT count(DISTINCT ord.order_id),dept.department 
        FROM order_products__train as ord,products as prod,departments as dept 
        WHERE ord.product_id=prod.product_id AND prod.department_id=dept.department_id 
        GROUP BY dept.department
        """):
    print(row)

(3105, 'alcohol')
(6362, 'babies')
(36424, 'bakery')
(61482, 'beverages')
(21879, 'breakfast')
(1323, 'bulk')
(29416, 'canned goods')
(87400, 'dairy eggs')
(32281, 'deli')
(25876, 'dry goods pasta')
(51071, 'frozen')
(21647, 'household')
(9716, 'international')
(24286, 'meat seafood')
(6933, 'missing')
(1725, 'other')
(47599, 'pantry')
(14908, 'personal care')
(2729, 'pets')
(96927, 'produce')
(57302, 'snacks')


Получим количество заказов в разбивке по дням недели:

In [27]:
for row in cursor.execute("""
    select 
      count(order_id) as total_orders, 
      (case 
         when order_dow = '0' then 'Sunday'
         when order_dow = '1' then 'Monday'
         when order_dow = '2' then 'Tuesday'
         when order_dow = '3' then 'Wednesday'
         when order_dow = '4' then 'Thursday'
         when order_dow = '5' then 'Friday'
         when order_dow = '6' then 'Saturday'              
       end) as day_of_week 
      from orders  
     group by order_dow 
     order by total_orders desc
     """):
    print(row)

(600905, 'Sunday')
(587478, 'Monday')
(467260, 'Tuesday')
(453368, 'Friday')
(448761, 'Saturday')
(436972, 'Wednesday')
(426339, 'Thursday')


Получим количество заказов в разбивке по часам дня:

In [28]:
for row in cursor.execute("""
    select 
      count(order_id) as total_orders, 
      order_hour_of_day as hour 
      from orders 
     group by order_hour_of_day 
     order by order_hour_of_day
          """):
    print(row)

(22758, '00')
(12398, '01')
(7539, '02')
(5474, '03')
(5527, '04')
(9569, '05')
(30529, '06')
(91868, '07')
(178201, '08')
(257812, '09')
(288418, '10')
(284728, '11')
(272841, '12')
(277999, '13')
(283042, '14')
(283639, '15')
(272553, '16')
(228795, '17')
(182912, '18')
(140569, '19')
(104292, '20')
(78109, '21')
(61468, '22')
(40043, '23')


Получим 10 наиболее популярных продуктов:

In [29]:
for row in cursor.execute("""
    select count(opp.order_id) as orders, p.product_name as popular_product
    from order_products__train opp, products p
    where p.product_id = opp.product_id 
    group by popular_product 
    order by orders desc 
    limit 10
    """):
    print(row)

(18726, 'Banana')
(15480, 'Bag of Organic Bananas')
(10894, 'Organic Strawberries')
(9784, 'Organic Baby Spinach')
(8135, 'Large Lemon')
(7409, 'Organic Avocado')
(7293, 'Organic Hass Avocado')
(6494, 'Strawberries')
(6033, 'Limes')
(5546, 'Organic Raspberries')


In [30]:
conn.close()

### Загрузка данных SQLite в датафрейм

Установим соединение в базой данных и загрузим таблицу `departments` в датафрейм:

In [31]:
conn = sqlite3.connect('instacart.db')

df_dept = pd.read_sql_query("SELECT * FROM departments", conn)
df_dept

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


Можно сделать столбец `department_id` индексом строк:

In [32]:
df_dept = df_dept.set_index('department_id')
df_dept.head()

,department
department_id,
1,frozen
2,other
3,bakery
4,produce
5,alcohol


Возможны и другие варианты загрузки данных:

In [33]:
cur_aisles = conn.execute("SELECT * From aisles")
cur_aisles.description

(('aisle_id', None, None, None, None, None, None),
 ('aisle', None, None, None, None, None, None))

In [34]:
cols = [col[0] for col in cur_aisles.description]
df_aisles = pd.DataFrame.from_records(data = cur_aisles.fetchall(), 
                                      columns = cols, index=['aisle_id'])
df_aisles

,aisle
aisle_id,
1,prepared soups salads
2,specialty cheeses
3,energy granola bars
4,instant foods
5,marinades meat preparation
...,...
130,hot cereal pancake mixes
131,dry pasta
132,beauty


### Слияние датафреймов

**Аргументы функции merge**


| Аргумент        | Описание
| ------------- |:-------------:|
|left|Объект DataFrame в левой части операции слияния|
|right|Объект DataFrame в правой части операции слияния|
|how|Допустимые значения: 'inner', 'outer', 'left', 'right '|
|on|Имена столбцов, по которым производится соединение. Должны присутствовать в обоих объектах DataFrame. Если не заданы и не указаны никакие другие ключи соединения, то используются имена столбцов, общих для обоих объектов|
|left_on|Столбцы левого DataFrame, используемые как ключи соединения|
|right_on|Столбцы правого DataFrame, используемые как ключи соединения|
|left index|Использовать индекс строк левого DataFrame в качестве его ключа соединения (или нескольких ключей в случае мультииндекса)|
|right_index|То же, что left_index, но для правого DataFrame|
|sort|Сортировать слитые данные лексикографически по ключам соединения по умолчанию True. Иногда при работе с большими наборами данных лучше отключить
|suffixes|Кортеж строк, которые дописываются в конец совпадающих имен столбцов;
|сору|Если равен False, то в некоторых особых случаях разрешается не копировать данные в результирующую структуру.|

In [35]:
df1 = pd.DataFrame({'key': ['a','a', 'a', 'b', 'b', 'c', 'd'],
                    'data1': range(7)},
                    index = range(3,10))
df1

,key,data1
3,a,0
4,a,1
5,a,2
6,b,3
7,b,4
8,c,5
9,d,6


In [36]:
df2 = pd.DataFrame({'key': ['a', 'b', 'b', 'f'],
                    'data2': range(4)})
df2

,key,data2
0,a,0
1,b,1
2,b,2
3,f,3


По умолчанию слияние датафреймов работает по столбцу (столбцам) с повторяющимися именами (аналогично методу `INNER JOIN` в SQL):

In [37]:
pd.merge(df1, df2)

,key,data1,data2
0,a,0,0
1,a,1,0
2,a,2,0
3,b,3,1
4,b,3,2
5,b,4,1
6,b,4,2


Аналогичный эффект достигался бы для кода:

`pd.merge(df1, df2, on='key')`

При слиянии можно использовать разные имена столбцов:

In [38]:
pd.merge(df1, df2, left_on='data1', right_on='data2')

,key_x,data1,key_y,data2
0,a,0,a,0
1,a,1,b,1
2,a,2,b,2
3,b,3,f,3


Можно задать другие суффиксы для повторяющихся имен столбцов:

In [39]:
pd.merge(df1, df2, left_on='data1', right_on='data2',suffixes=('_left', '_right'))

,key_left,data1,key_right,data2
0,a,0,a,0
1,a,1,b,1
2,a,2,b,2
3,b,3,f,3


Можно выполнить соединение по нескольким столбцам:

In [40]:
pd.merge(df1, df2, left_on=['data1', 'key'], right_on=['data2', 'key'])

,key,data1,data2
0,a,0,0


Можно изменить тип соединения по умолчанию (`INNER JOIN`) на другой:

In [41]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,a,0.0,0.0
1,a,1.0,0.0
2,a,2.0,0.0
3,b,3.0,1.0
4,b,3.0,2.0
5,b,4.0,1.0
6,b,4.0,2.0
7,c,5.0,NaN
8,d,6.0,NaN
9,f,NaN,3.0


Также датафреймы могут соединяться не по значениям в столбцах, а по индексам строк:

In [42]:
pd.merge(df1, df2, left_index=True, right_on='data2')

,key_x,data1,key_y,data2
3,a,0,f,3


In [43]:
pd.merge(df1, df2, left_on='data1', right_index=True)

,key_x,data1,key_y,data2
3,a,0,a,0
4,a,1,b,1
5,a,2,b,2
6,b,3,f,3


In [44]:
pd.merge(df1, df2, left_index=True, right_index=True)

,key_x,data1,key_y,data2
3,a,0,f,3


### Конкатенация датафреймов

Рассмотрим конкатенацию на следующих датафреймах:

In [45]:
df1 = pd.DataFrame(index = range(3), data = np.arange(9).reshape(3, 3), 
                   columns=['a', 'b', 'c'])
df1

,a,b,c
0,0,1,2
1,3,4,5
2,6,7,8


In [46]:
df2 = pd.DataFrame(index = range(2,5), data = np.arange(9, 18).reshape(3, 3), 
                   columns=['a', 'c', 'd'])
df2

,a,c,d
2,9,10,11
3,12,13,14
4,15,16,17


По умолчанию получим:

In [47]:
pd.concat([df1, df2]) 

,a,b,c,d
0,0,1.0,2,NaN
1,3,4.0,5,NaN
2,6,7.0,8,NaN
2,9,NaN,10,11.0
3,12,NaN,13,14.0
4,15,NaN,16,17.0


Чтобы оставить только повторяющиеся столбцы, укажем параметр `join='inner'`:

In [48]:
pd.concat([df1, df2], axis=0, join='inner')

,a,c
0,0,2
1,3,5
2,6,8
2,9,10
3,12,13
4,15,16


Чтобы выполнить конкатенацию по оси столбцов, изменим ось:

In [49]:
pd.concat([df1, df2], axis=1)

,a,b,c,a,c,d
0,0.0,1.0,2.0,NaN,NaN,NaN
1,3.0,4.0,5.0,NaN,NaN,NaN
2,6.0,7.0,8.0,9.0,10.0,11.0
3,NaN,NaN,NaN,12.0,13.0,14.0
4,NaN,NaN,NaN,15.0,16.0,17.0


### Группировка данных

Рассмотрим имеющиеся в Pandas механизмы группировки данных на следующем датафрейме:

In [50]:
df = pd.DataFrame({'x': ['a','a','b','b','c','c'],
                   'y': [2,4,0,5,5,10]})
df

,x,y
0,a,2
1,a,4
2,b,0
3,b,5
4,c,5
5,c,10


Чтобы выполнить группировку по значениям в столбце `x` создадим следующий объект:

In [51]:
groups = df.groupby(['x'])
groups

Из объекта можно получить информацию 
* о количестве групп, которые будут созданы:

In [52]:
groups.ngroups

3

* о количестве элементов в каждой группе:

In [53]:
groups.size()

x
a    2
b    2
c    2
dtype: int64

* о том, что представляют из себя найденные группы с индексами строк:

In [54]:
groups.groups

{'a': [0, 1], 'b': [2, 3], 'c': [4, 5]}

Можно получить данные конкретной группы:

In [55]:
groups.get_group('b')

,x,y
2,b,0
3,b,5


Можно извлечь нужную строку (например, с первым индексом) из каждой группы:

In [56]:
groups.nth([1])

,y
x,
a,4
b,5
c,10


Обход групп можно организовать в цикле:

In [57]:
for key, group in groups:
    print('** ', key, ' **')
    print(group) 
    print('-'*9)

**  a  **
   x  y
0  a  2
1  a  4
---------
**  b  **
   x  y
2  b  0
3  b  5
---------
**  c  **
   x   y
4  c   5
5  c  10
---------


В группах возможны вычисления, например, вычисление среднего:

In [58]:
groups.y.mean()

x
a    3.0
b    2.5
c    7.5
Name: y, dtype: float64

## Построение популярных наборов и ассоциативных правил

Модуль `itertools` расширяет функционал Python, связанный с созданием последовательностей объектов и манипулированием ими.

Функция `combinations()` позволяет комбинировать отдельные элементы последовательности и принимает два аргумента. Первый позволяет задать определенный объект, а второй – количество значений, которые будут присутствовать в каждом новом элементе. 

In [59]:
from itertools import combinations
data = list(combinations('РУДН', 2))
print(data)

[('Р', 'У'), ('Р', 'Д'), ('Р', 'Н'), ('У', 'Д'), ('У', 'Н'), ('Д', 'Н')]


Как видно из кода, функция получает строку `РУДН`, которая впоследствии раскладывается на отдельные символы. Далее происходит группировка по 2 буквы так, чтобы каждая новая выборка отличалась от всех существующих. Функция print выводит полученный список data на экран, отображая все сформированные пары символов `Р`, `У`, `Д`, `Н`.

Аналогично функция combinations работает и со списками:

In [60]:
data2 = list(combinations([1,2,3,4,5], 3))
print(data2)

[(1, 2, 3), (1, 2, 4), (1, 2, 5), (1, 3, 4), (1, 3, 5), (1, 4, 5), (2, 3, 4), (2, 3, 5), (2, 4, 5), (3, 4, 5)]


Функция `chain()` выполняет объединение списков, возвращая итератор, как это показано в следующем примере. Итоговый массив содержит все элементы данных последовательностей.

In [61]:
from itertools import chain
list1 = ['Р', 'У', 'Д', 'Н']
list2 = [1, 2, 3, 4, 5]
data = list(chain(list1, list2))
print(data)

['Р', 'У', 'Д', 'Н', 1, 2, 3, 4, 5]


Функция `chain.from_iterable()` работает аналогично функции `chain()`, выполняя объединение списков. Отличие заключается в том, что аргумент только один – вложенный список со списками, которые надо объединить.

In [62]:
from itertools import chain
list0 = [['Р', 'У', 'Д', 'Н'], [1, 2, 3, 4, 5]]
data = list(chain.from_iterable(list0))
print(data)

['Р', 'У', 'Д', 'Н', 1, 2, 3, 4, 5]


Для построения __множества всех подмножеств__ данного множества (в теоретико-множественном смысле) можем использовать следующую функцию:

In [63]:
from itertools import chain, combinations

def powerset(iterable):
    xs = list(iterable)
    # возвращаем итератор, а не список
    return chain.from_iterable(combinations(xs,n) for n in range(len(xs)+1))

print(list(powerset([1,2,3,4])))

[(), (1,), (2,), (3,), (4,), (1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4), (1, 2, 3), (1, 2, 4), (1, 3, 4), (2, 3, 4), (1, 2, 3, 4)]


Будем использовать следующий учебный набор транзакций, в котором набор предметов в транзакции представляется в качестве множества:

In [64]:
D_train = [
    [ 1, {"A","B","D","E"} ],
    [ 2, {"B","C","E"} ],
    [ 3, {"A","B","D","E"} ],
    [ 4, {"A","B","C","E"} ],
    [ 5, {"A","B","C","D","E"} ],
    [ 6, {"B","C","D"} ],
]

Чтобы посчитать поддержку заданного набора предметов, будем использовать следуюшую функцию:

In [65]:
def ComputeSupport( X, D ):
    supX = 0
    for _,itemset in D:
        if X.issubset( itemset ):
            supX += 1
    return supX

Например,

In [66]:
X = {"B","C","E"}

print("\nsup(", X, ") =", ComputeSupport(X, D_train))


sup( {'C', 'B', 'E'} ) = 3


Для построения множества всех подмножеств используем функцию powerset():

In [67]:
I = {"A","B","C", "D", "E"}

print("\nМножество всех подмножеств множества", I, ":")  
for itemset in powerset( I ):
    print( itemset )


Множество всех подмножеств множества {'C', 'A', 'E', 'B', 'D'} :
()
('C',)
('A',)
('E',)
('B',)
('D',)
('C', 'A')
('C', 'E')
('C', 'B')
('C', 'D')
('A', 'E')
('A', 'B')
('A', 'D')
('E', 'B')
('E', 'D')
('B', 'D')
('C', 'A', 'E')
('C', 'A', 'B')
('C', 'A', 'D')
('C', 'E', 'B')
('C', 'E', 'D')
('C', 'B', 'D')
('A', 'E', 'B')
('A', 'E', 'D')
('A', 'B', 'D')
('E', 'B', 'D')
('C', 'A', 'E', 'B')
('C', 'A', 'E', 'D')
('C', 'A', 'B', 'D')
('C', 'E', 'B', 'D')
('A', 'E', 'B', 'D')
('C', 'A', 'E', 'B', 'D')


Построение популярных наборов предметов с заданным уровнем минимальной поддержки может быть реализовано при помощи алгоритма __BruteForce__ (полного перебора):

In [68]:
def BruteForce( D, I, minsup ):
    F = []
    for X in powerset( I ):
        if len( X ) > 0:
            supX = ComputeSupport( set( X ), D )
            if supX >= minsup:
                F.append( [ X, supX ] )
    return F

Построим популярные наборы в учебной наборе транзакций для заданного уровня минимальной поддержки:

In [69]:
minsup = 3

print("\nПопулярные наборы предметов для minsup =", minsup, ":") 

for itemset in BruteForce( D_train, I, minsup ):
    print( itemset )


Популярные наборы предметов для minsup = 3 :
[('C',), 4]
[('A',), 4]
[('E',), 5]
[('B',), 6]
[('D',), 4]
[('C', 'E'), 3]
[('C', 'B'), 4]
[('A', 'E'), 4]
[('A', 'B'), 4]
[('A', 'D'), 3]
[('E', 'B'), 5]
[('E', 'D'), 3]
[('B', 'D'), 4]
[('C', 'E', 'B'), 3]
[('A', 'E', 'B'), 4]
[('A', 'E', 'D'), 3]
[('A', 'B', 'D'), 3]
[('E', 'B', 'D'), 3]
[('A', 'E', 'B', 'D'), 3]


Замечание. При построении дерева префиксов в алгоритмах Apriori, Eclat и dEclat нужно разделять названия предметов (товаров) каким-либо символом (разделителем), который не встречается в названиях.

Будем считать, что популярный набор предметов задан (построен) и задача состоит в получении ассоциативных правил с заданным минимальным уровнем достоверности.  

In [70]:
F_set, _ = BruteForce( D_train, I, minsup )[-1]
F_set

('A', 'E', 'B', 'D')

Алгоритм AssociationRules для случая построения ассоциативных правил по заданному популярному набору предметов может быть реализован следующим образом:

In [ ]:
def powersetk(iterable,k):
    xs = list(iterable)
    # возвращаем итератор, а не список
    return chain.from_iterable(combinations(xs,n) for n in range(k,len(xs)+1))

In [ ]:
list(powersetk(I,4))

In [ ]:
def AssociationRules(D, Z_set, minconf):
    A_rules = []
    supZ = ComputeSupport(set(Z_set), D)
    A_set = list(powersetk(Z_set,1))[:-1]
    while len(A_set)>0:
        X_set = A_set[-1]
        A_set.pop()
        conf = supZ/ComputeSupport(set(X_set), D)
        if conf >= minconf:
            Y_set = sorted(list(set(Z_set)-set(X_set)))
            A_rules.append([X_set, Y_set, supZ, conf])
        else:
            for W_set in powersetk(X_set,1):
                if W_set in A_set:
                    A_set.remove(W_set)
    return A_rules

In [ ]:
AssociationRules(D_train, F_set, 0.4)

## Поиск ассоциативных правил при помощи `mlxtend`

В библиотеке машинного обучения `scikit-learn` алгоритмы поиска ассоциативных правил не реализованы, однако некоторые алгоритмы реализованы в библиотеке `mlxtend` (установка командой `pip install mlxtend`). 

В библиотеке `mlxtend` база транзакций должна быть представлена в форме бинарной базы (по строкам транзакции, по столбцам предметы (товары)).

Пусть исходные набор данных имеет вид транзакционной (горизонтальной) базы (без номеров транзакций): 

In [ ]:
dataset = [['Milk', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Dill', 'Onion', 'Nutmeg', 'Kidney Beans', 'Eggs', 'Yogurt'],
           ['Milk', 'Apple', 'Kidney Beans', 'Eggs'],
           ['Milk', 'Unicorn', 'Corn', 'Kidney Beans', 'Yogurt'],
           ['Corn', 'Onion', 'Onion', 'Kidney Beans', 'Ice cream', 'Eggs']]

Для преобразования транзакционной базы в бинарную базу (датафрейм) в пакете `mlxtend` имеется соответствующий инструментарий:

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
dataset_bin = te.fit(dataset).transform(dataset)
df = pd.DataFrame(dataset_bin, columns=te.columns_)
df

Для поиска популярных наборов предметов в `mlxtend` реализованы алгоритмы `apriori`, `fpgrowth` и `fpmax`:

In [ ]:
from mlxtend.frequent_patterns import apriori

apriori(df, min_support=0.6)

Чтобы получить популярные наборы предметов с названиями предметов вместо номеров, можно указать ключ `use_colnames=True`:

In [ ]:
itemsets_apr = apriori(df, min_support=0.6, use_colnames=True)
itemsets_apr

In [ ]:
from mlxtend.frequent_patterns import fpgrowth

fpgrowth(df, min_support=0.6)

In [ ]:
itemsets_fpg = fpgrowth(df, min_support=0.6, use_colnames=True)
itemsets_fpg

In [ ]:
itemsets_fpg.dtypes

In [ ]:
from mlxtend.frequent_patterns import fpmax

fpmax(df, min_support=0.6)

In [ ]:
itemsets_fpm = fpmax(df, min_support=0.6, use_colnames=True)
itemsets_fpm

Чтобы по известному популярному набору предметов построить ассоциативные правила, можно воспользоваться функцией `association_rules`:

In [ ]:
from mlxtend.frequent_patterns import association_rules

association_rules(itemsets_apr, metric="confidence", min_threshold=0.7)

### Задание на лабораторную работу №2

#### Задание (10 баллов)

Для закрепленного за Вами варианта лабораторной работы:

1.	При помощи модуля sqlite3 откройте базу данных Instacart в файле instacart.db.

2.  Загрузите таблицы `departments` и `products` в датафреймы Pandas. При помощи запроса SELECT извлеките из таблицы `order_products__train` записи, соответствующие указанным в индивидуальном задании дню недели (поле `order_dow` таблицы `orders`) и коду департамента (поле `department_id` таблицы `products`) и загрузите в датафрейм Pandas. Определите количество строк в полученном датафрейме и определите количество товаров (стоолбец `product_id`) в транзакциях датафрейма.

3.	Определите пять наиболее популярных товаров в датафрейме транзакций и определите количество покупок (транзакций) этих товаров.

4.	Постройте транзакционную базу данных из полученного датафрейма, используя в качестве идентификатора транзакции столбец `order_id`, а в качестве названий товаров - поле `product_name` из датафрейма для таблицы `products`, соответствующее столбцу `product_id`. Найдите в транзакционной базе данных три транзакции с наибольшим количеством товаров и выведите их на экран. 

5.	Постройте по транзакционной базе данных бинарную базу данных в формате датафрейма пакета `mlxtend`. По бинарной базе данных определите пять  наиболее популярных товаров и определите количество покупок (транзакций) этих товаров.

6.	При помощи указанного в индивидуальном задании метода построения популярных наборов предметов постройте популярный набор предметов с минимальной поддержкой не менее 3, имеющий максимальную длину. При отсутствии таких наборов уменьшите поддержку до 2. В случае нехватки вычислительных ресурсов (слишком долгой работы программы) при построении популярных наборов предметов сокращайте число записей в наборе данных (например, делая выборку половины записей набора).

7.	Используя пакет `mlxtend` или реализацию на Python, постройте набор ассоциативных правил для полученного популярного наборов предметов. Используйте уровень достоверности (confidence), равный 0.6.

8.	Для построенного набора ассоциативных правил вычислите показатель (меру) оценки ассоциативных правил, указанную в индивидуальном задании, и определите ассоциативные правила с наилучшим значением показателя оценки.
